### 한국수출입은행 - 국제금리 (기간 지정)

In [1]:
date_from = "2023-06-01"
date_to = "2023-07-04"
url = 'https://www.koreaexim.go.kr/site/program/financial/internationalJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

In [4]:
import requests
import pandas as pd

date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_rate_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP03'             # AP03 : 국제금리
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    
    dict_data = response.json()
    if dict_data['result'] != RESULT_SUCCESS:
        print('Result is not successful')

    for key in dict_data.keys():
        if key in ['result', 'cur_fund', 'sfln_intrc_nm', 'int_r']:
            pass
        else:
            df_rate = pd.json_normalize(dict_data.get(key))
            if not df_rate.empty:
                df_rate.drop('result', axis=1, inplace=True)
                df_rate.insert(0, 'date', date_str)
                df_rate_all = pd.concat([df_rate_all, df_rate], ignore_index=True)

In [5]:
df_rate_all.shape

(1279, 13)

In [11]:
df_rate_all.dropna(subset=['int_r'], how='any', axis=0, inplace=True)
df_rate_all = df_rate_all[['date', 'cur_fund', 'sfln_intrc_nm', 'int_r']].sort_values(['date', 'cur_fund'])
df_rate_all.reset_index(drop=True, inplace=True)
df_rate_all

,date,cur_fund,sfln_intrc_nm,int_r
0,20230601,EUR,ESTR 익일물,3.148
1,20230601,EUR,EURIBOR1개월,3.214
2,20230601,EUR,EURIBOR3개월,3.474
3,20230601,EUR,EURIBOR6개월,3.77
4,20230601,EUR,EURIBOR12개월,3.965
...,...,...,...,...
1021,20230704,USD,SWAP금리 1년,5.656
1022,20230704,USD,SWAP금리 2년,5.165
1023,20230704,USD,5년이하,4.82
1024,20230704,USD,5년초과8.5년이하,4.59


In [12]:
df_rate_all.groupby(['date', 'cur_fund']).agg(n = ('cur_fund', 'count'))

n
date     cur_fund    
20230601 EUR       14
         JPY       10
         USD       14
20230602 EUR       14
         JPY       11
...                ..
20230703 JPY       11
         USD       14
20230704 EUR       14
         JPY       10
         USD       14

[102 rows x 1 columns]

In [13]:
currency = 'USD'
df_rate_all.query('cur_fund == @currency')

,date,cur_fund,sfln_intrc_nm,int_r
24,20230601,USD,SOFR 익일물,5.06
25,20230601,USD,SWAP금리(RFR) 6개월,5.2639
26,20230601,USD,SWAP금리(RFR) 1년,5.0557
27,20230601,USD,SWAP금리(RFR) 2년,4.3099
28,20230601,USD,LIBOR1개월,5.17043
...,...,...,...,...
1021,20230704,USD,SWAP금리 1년,5.656
1022,20230704,USD,SWAP금리 2년,5.165
1023,20230704,USD,5년이하,4.82
1024,20230704,USD,5년초과8.5년이하,4.59
